In [1]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from folium import IFrame
import ipywidgets
#import geopy
#import geocoder

import numpy as np
import pandas as pd

In [2]:
centros_medicos_clean = pd.read_csv("../data/centros_medicos_clean.csv")
centros_medicos_clean.head()

,Unnamed: 0,NOMBRE,DIRECCION,DISTRITO,COD_POSTAL,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TIPO
0,0,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,CHAMARTIN,28002,443159,4477482,40.446053,-3.670277,CentrosAsistenciaDrogodependientes
1,1,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL 0,LATINA,28011,438577,4474219,40.416331,-3.723991,CentrosAsistenciaDrogodependientes
2,2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,ARGANZUELA,28045,440757,4471619,40.393057,-3.698067,CentrosAsistenciaDrogodependientes
3,3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,HORTALEZA,28043,446072,4479929,40.468289,-3.636150,CentrosAsistenciaDrogodependientes
4,4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,LATINA,28047,435820,4471982,40.395971,-3.756272,CentrosAsistenciaDrogodependientes


## Mapa normal

In [3]:
max_lat = centros_medicos_clean['LATITUD'].max()
max_lat

40.540270273154896

In [4]:
max_lon = centros_medicos_clean['LONGITUD'].max()
max_lon

-3.57810735476017

In [5]:
map_centros = Map(location=[max_lat, max_lon],zoom_start=10)
map_centros

In [6]:
for i,row in centros_medicos_clean.iterrows():
    
    tipos = {"location" : [row["LATITUD"], row["LONGITUD"]],
                "tooltip" : row["TIPO"]}
    
    if row["TIPO"] == "CentrosAsistenciaDrogodependientes":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "heartbeat",
                    icon_color = "red"
        )

    elif row["TIPO"] == "OtrosCentrosMedicos":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "plus-square",
                    icon_color = "blue"
        )
     
    elif row["TIPO"] == "CentrosEspecialidadesMedicas":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "medkit",
                    icon_color = "black"
        )
    
    elif row["TIPO"] == "CentrosSalud":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "h-square",
                    icon_color = "blue"
        )
    
    elif row["TIPO"] == "CentrosRehabilitacionPsicosocial":
        icon = Icon( color = "red",
                    prefix = "fa",
                    icon = "heart-o",
                    icon_color = "white"
        )
    
    elif row["TIPO"] == "CentrosPrevencionEnfermedades":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "stethoscope",
                    icon_color = "green"
        )
        
    elif row["TIPO"] == "CentrosSaludMental":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "user-md",
                    icon_color = "black"
        )
    
    # HospitalesClinicasSanatorios
    else:
        icon = Icon(color = "white",
                    prefix = "fa",
                    icon = "hospital-o",
                    icon_color = "gray"
        )
    
    Marker (**tipos,icon = icon).add_to(map_centros)

folium.LayerControl(collapsed = False).add_to(map_centros) 
map_centros

## Mapa de calor

In [7]:
centros_medicos_clean['TIPO'].unique()

array(['CentrosAsistenciaDrogodependientes', 'OtrosCentrosMedicos',
       'CentrosEspecialidadesMedicas', 'CentrosSalud',
       'CentrosRehabilitacionPsicosocial',
       'CentrosPrevencionEnfermedades', 'CentrosSaludMental',
       'HospitalesClinicasSanatorios'], dtype=object)

In [9]:
def dataframescapas(elemento):
    return centros_medicos_clean[centros_medicos_clean['TIPO'].str.contains(f"{elemento}")]

In [10]:
capas = list(centros_medicos_clean['TIPO'].unique())
capas

['CentrosAsistenciaDrogodependientes',
 'OtrosCentrosMedicos',
 'CentrosEspecialidadesMedicas',
 'CentrosSalud',
 'CentrosRehabilitacionPsicosocial',
 'CentrosPrevencionEnfermedades',
 'CentrosSaludMental',
 'HospitalesClinicasSanatorios']

In [12]:
data0 = dataframescapas(capas[0])
data1 = dataframescapas(capas[1])
data2 = dataframescapas(capas[2])
data3 = dataframescapas(capas[3])
data4 = dataframescapas(capas[4])
data5 = dataframescapas(capas[5])
data6 = dataframescapas(capas[6])
data7 = dataframescapas(capas[7])

In [13]:
map_capas = Map(location=[max_lat, max_lon],zoom_start=10)

In [14]:
data0_group = folium.FeatureGroup(name = "Drogodependientes")
HeatMap(data = data0[["LATITUD","LONGITUD"]], radius = 10).add_to(data0_group)
data0_group.add_to(map_capas)


data1_group = folium.FeatureGroup(name = "Otros Centos Medicos")
HeatMap(data = data1[["LATITUD","LONGITUD"]], radius = 10).add_to(data1_group)
data1_group.add_to(map_capas)


data2_group = folium.FeatureGroup(name = "Especialidades Medicas")
HeatMap(data = data2[["LATITUD","LONGITUD"]], radius = 10).add_to(data2_group)
data2_group.add_to(map_capas)

data3_group = folium.FeatureGroup(name = "Centros Salud")
HeatMap(data = data3[["LATITUD","LONGITUD"]], radius = 10).add_to(data3_group)
data3_group.add_to(map_capas)


data4_group = folium.FeatureGroup(name = "Centros Rehabilitacion Psicosocial")
HeatMap(data = data4[["LATITUD","LONGITUD"]], radius = 10).add_to(data4_group)
data4_group.add_to(map_capas)


data5_group = folium.FeatureGroup(name = "Centros Prevencion de Enfermedades")
HeatMap(data = data5[["LATITUD","LONGITUD"]], radius = 10).add_to(data5_group)
data5_group.add_to(map_capas)


data6_group = folium.FeatureGroup(name = "Centros Salud Mental")
HeatMap(data = data6[["LATITUD","LONGITUD"]], radius = 10).add_to(data6_group)
data6_group.add_to(map_capas)


data7_group = folium.FeatureGroup(name = "Hospitales Clinicas Sanatorios")
HeatMap(data = data7[["LATITUD","LONGITUD"]], radius = 10).add_to(data7_group)
data7_group.add_to(map_capas)

# Esta ultima linea SOLO se pone alfinal de haber definido tus capas, para que te aparezca el Later control con todas tus opciones

folium.LayerControl(collapsed = False).add_to(map_capas) 



map_capas

## Tipos de mapas

In [ ]:
map_ = Map(location=[max_lat, max_lon],zoom_start=10, control_scale = True)
map_

In [ ]:
select_widget = ipywidgets.Select(
    options = ['Open Street Map', 'Terrain', 'Toner', 'Watercolor', 'Positron', 'Dark Matter'],
    value = 'Open Street Map', 
    description = 'Map type:',
    disabled = False)

def select(map_type):
    if map_type == 'Open Street Map':
        display(folium.Map(location=[max_lat, max_lon], zoom_start=10, control_scale = True))
        
    elif map_type == 'Terrain':
        display(folium.Map(location=[max_lat, max_lon], tiles = 'Stamen Terrain', zoom_start=10, control_scale = True))
    
    elif map_type == 'Toner':
        display(folium.Map(location=[max_lat, max_lon], tiles = 'Stamen Toner', zoom_start=10, control_scale = True))
    
    elif map_type == 'Watercolor':
        display(folium.Map(location=[max_lat, max_lon], tiles = 'Stamen Watercolor', zoom_start=10, control_scale = True))
    
    elif map_type == 'Positron':
        display(folium.Map(location=[max_lat, max_lon], tiles = 'CartoDB Positron', zoom_start=10, control_scale = True))

    elif map_type == 'Dark Matter':
        display(folium.Map(location=[max_lat, max_lon], tiles = 'CartoDB Dark_Matter', zoom_start=10, control_scale = True))
    
ipywidgets.interact(select, map_type = select_widget)

In [ ]:
map_layer_control = Map(location=[max_lat, max_lon],zoom_start=10, control_scale = True)


folium.raster_layers.TileLayer('Open Street Map').add_to(map_layer_control)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map_layer_control)
folium.raster_layers.TileLayer('Stamen Toner').add_to(map_layer_control)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(map_layer_control)
folium.raster_layers.TileLayer('CartoDB Positron').add_to(map_layer_control)
folium.raster_layers.TileLayer('CartoDB Dark_Matter').add_to(map_layer_control)


folium.LayerControl().add_to(map_layer_control)
map_layer_control